In [1]:
import os
import sys
import warnings
import pandas as pd
from tqdm.notebook import tqdm

base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"base_path: {base_path}")
sys.path.append(base_path)

base_path: /home/toc3/sr-press


In [2]:
import express.config as config
from express.databases import SQLiteDatabase
from express.datasets import PressingDataset
from express.utils import add_names

In [3]:
TRAIN_DB_PATH = os.path.join(base_path, "stores/train_database.sqlite")
TEST_DB_PATH = os.path.join(base_path, "stores/test_database.sqlite")

train_db = SQLiteDatabase(TRAIN_DB_PATH)
test_db = SQLiteDatabase(TEST_DB_PATH)

print("train_db:", train_db)
print("test_db:", test_db)

train_db: <express.databases.sqlite.SQLiteDatabase object at 0x7f3b93dcd0d0>
test_db: <express.databases.sqlite.SQLiteDatabase object at 0x7f3b94242e20>


In [4]:
game_id = 3895302

df_actions = add_names(train_db.actions(game_id))
df_actions

original_event_id  period_id  \
game_id action_id                                                    
3895302 0          221b0c8d-6386-4ae8-bb4a-a1dc98742312          1   
        1          77809242-1460-4395-8779-94a0cfc275b1          1   
        2          ff56e821-21e9-4cef-ba2a-7eb5eb3769c6          1   
        3          cf9088bc-7e59-4d57-8ac5-31658da858bb          1   
        4          4464cb75-f45f-4508-8444-2560d1625d06          1   
...                                                 ...        ...   
        3025       4e61f75d-564b-4478-a2f0-4b7e20e8600e          2   
        3026                                       None          2   
        3027       91466764-aec7-4fac-a024-dd3deee3c737          2   
        3028       e75b1a76-8aca-461f-846f-70b8b9e35fa9          2   
        3029       dcc0c591-7ebd-4e42-8206-04bfefa94f33          2   

                   time_seconds  team_id  player_id  start_x  start_y  \
game_id action_id                                                       
3895302 0                3.4170      176      34870  52.0625   33.660   
        1                3.8700      176      12299  53.8125   34.340   
        2                4.7320      176      12299  53.8125   34.085   
        3                6.7280      176      31100  74.7250   36.295   
        4                7.6220      176      31100  74.7250   36.720   
...                         ...      ...        ...      ...      ...   
        3025          2661.1130      904      10336  57.4000   45.220   
        3026          2661.7455      176      24179  55.1250   44.795   
        3027          2662.5800      904      40724  77.7875   21.335   
        3028          2662.8440      176      24179  84.0875   19.890   
        3029          2665.0790      904      40724  92.6625   26.945   

                      end_x    end_y  bodypart_id  ...  possession_team_id  \
game_id action_id                                  ...                       
3895302 0           53.8125  34.3400            5  ...               176.0   
        1           53.8125  34.0850            0  ...               176.0   
        2           74.7250  36.2950            5  ...               176.0   
        3           74.7250  36.7200            0  ...               176.0   
        4           76.3875  53.8050            5  ...               176.0   
...                     ...      ...          ...  ...                 ...   
        3025        77.7875  21.3350            4  ...               904.0   
        3026        84.0875  19.8900            0  ...                 NaN   
        3027        92.6625  26.9450            0  ...               904.0   
        3028        84.0875  19.8900            0  ...               904.0   
        3029       104.5625  35.9975            5  ...               904.0   

                   play_pattern_name  under_pressure  \
game_id action_id                                      
3895302 0              From Kick Off               0   
        1              From Kick Off               0   
        2              From Kick Off               0   
        3              From Kick Off               0   
        4              From Kick Off               0   
...                              ...             ...   
        3025            From Counter               0   
        3026                    None               0   
        3027            From Counter               1   
        3028            From Counter               0   
        3029            From Counter               0   

                                                               extra  \
game_id action_id                                                      
3895302 0          {'pass': {'recipient': {'id': 12299, 'name': '...   
        1                  {'carry': {'end_location': [59.0, 40.6]}}   
        2          {'pass': {'recipient': {'id': 31100, 'name': '...   
        3                  {'carry': {'end_location': [35.1, 43.7]}}   
 

In [14]:
has_required_players = df_actions["freeze_frame_360"].apply(lambda frame: (
    frame is not None and
    sum(player["teammate"] for player in frame) >= 3 and
    sum(not player["teammate"] for player in frame) >= 3
))

df_actions[has_required_players]

original_event_id  period_id  \
game_id action_id                                                    
3895302 2          ff56e821-21e9-4cef-ba2a-7eb5eb3769c6          1   
        3          cf9088bc-7e59-4d57-8ac5-31658da858bb          1   
        4          4464cb75-f45f-4508-8444-2560d1625d06          1   
        5          e9648006-8cd2-43f2-8615-c4cd3f583f05          1   
        6          cfa1f5e1-4e8e-4fc2-bbc8-4df5fdef8283          1   
...                                                 ...        ...   
        3022       b51cfe68-ebda-4c42-b315-79cdd9a3df5b          2   
        3023       6a3c49e3-5107-4e91-a5db-a20cb66156b8          2   
        3024       f908d853-a3e9-4ad7-99b4-6aef1f3e3286          2   
        3028       e75b1a76-8aca-461f-846f-70b8b9e35fa9          2   
        3029       dcc0c591-7ebd-4e42-8206-04bfefa94f33          2   

                   time_seconds  team_id  player_id  start_x  start_y  \
game_id action_id                                                       
3895302 2                 4.732      176      12299  53.8125   34.085   
        3                 6.728      176      31100  74.7250   36.295   
        4                 7.622      176      31100  74.7250   36.720   
        5                 9.037      176      51769  76.3875   53.805   
        6                 9.637      904      33401  74.3750   55.335   
...                         ...      ...        ...      ...      ...   
        3022           2654.465      176       8576  20.4750   48.195   
        3023           2656.038      904      10336  26.6000   49.980   
        3024           2660.647      176      12299  55.1250   44.795   
        3028           2662.844      176      24179  84.0875   19.890   
        3029           2665.079      904      40724  92.6625   26.945   

                      end_x    end_y  bodypart_id  ...  possession_team_id  \
game_id action_id                                  ...                       
3895302 2           74.7250  36.2950            5  ...               176.0   
        3           74.7250  36.7200            0  ...               176.0   
        4           76.3875  53.8050            5  ...               176.0   
        5           76.8250  55.0800            0  ...               176.0   
        6           74.3750  55.3350            0  ...               176.0   
...                     ...      ...          ...  ...                 ...   
        3022        20.1250  46.3250            4  ...               176.0   
        3023        57.4000  45.2200            0  ...               904.0   
        3024        55.1250  44.7950            0  ...               904.0   
        3028        84.0875  19.8900            0  ...               904.0   
        3029       104.5625  35.9975            5  ...               904.0   

                   play_pattern_name  under_pressure  \
game_id action_id                                      
3895302 2              From Kick Off               0   
        3              From Kick Off               0   
        4              From Kick Off               0   
        5              From Kick Off               1   
        6              From Kick Off               0   
...                              ...             ...   
        3022            Regular Play               0   
        3023            From Counter               1   
        3024            From Counter               0   
        3028            From Counter               0   
        3029            From Counter               0   

                                                               extra  \
game_id action_id                                                      
3895302 2          {'pass': {'recipient': {'id': 31100, 'name': '...   
        3                  {'carry': {'end_location': [35.1, 43.7]}}   
        4          {'pass': {'recipient': {'id': 51769, 'name': '...   
        5                  {'carry': {'end_location': [32.7, 65.3]}}   
 

In [11]:
df_actions["freeze_frame_360"]

game_id  action_id
3895302  0                                                         None
         1                                                         None
         2            [{'teammate': True, 'actor': False, 'keeper': ...
         3            [{'teammate': True, 'actor': True, 'keeper': F...
         4            [{'teammate': True, 'actor': False, 'keeper': ...
                                            ...                        
         3025                                                      None
         3026                                                      None
         3027                                                      None
         3028         [{'teammate': True, 'actor': False, 'keeper': ...
         3029         [{'teammate': False, 'actor': False, 'keeper':...
Name: freeze_frame_360, Length: 3030, dtype: object

In [5]:
from express import features as fs
from express import labels as ls

all_features = [f.__name__ for f in fs.all_features]
all_labels = [f.__name__ for f in ls.all_labels]
print("Features:", all_features)
print("Labels:", all_labels)

Features: ['actiontype', 'actiontype_onehot', 'result', 'result_onehot', 'bodypart', 'bodypart_onehot', 'time', 'startlocation', 'relative_startlocation', 'endlocation', 'relative_endlocation', 'startpolar', 'endpolar', 'movement', 'team', 'time_delta', 'space_delta', 'goalscore', 'angle', 'under_pressure', 'packing_rate', 'ball_height_onehot', 'player_possession_time', 'speed', 'nb_opp_in_path', 'dist_opponent', 'defenders_in_3m_radius', 'closest_3_players', 'extract_all_players', 'expected_receiver_and_presser']
Labels: ['concede_shots', 'counterpress']


In [6]:
import numpy as np
np.array([1,2,3])[1:3]

array([2, 3])

In [ ]:
train_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "filtered_datasets", "train"), # rename to datasets to filtered_datasets
    xfns=["expected_receiver_and_presser"],
    yfns=["counterpress"], #all_labels
    load_cached =False,
    nb_prev_actions = 3,
    min_players = 3,
)

test_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "filtered_datasets", "test"), # rename to datasets to filtered_datasets
    xfns=["expected_receiver_and_presser"],
    yfns=["counterpress"], 
    load_cached =False,
    nb_prev_actions = 3,
    min_players = 3,
)

In [8]:
train_dataset.create(train_db)
test_dataset.create(test_db)

Output()

Output()

Output()

Output()

In [9]:
train_dataset.features

teammate_opponent_distance1_a0  \
game_id action_id                                   
3788741 244                              1.726438   
        246                              3.416372   
        373                                   NaN   
        391                              1.297208   
        403                              2.343462   
...                                           ...   
3943043 2207                             1.105103   
        2230                             2.838836   
        2258                             4.990187   
        2283                            11.499700   
        2286                             2.046366   

                   teammate_opponent_distance2_a0  \
game_id action_id                                   
3788741 244                              3.613461   
        246                              5.454579   
        373                                   NaN   
        391                              3.243787   
        403                              5.037151   
...                                           ...   
3943043 2207                             1.798191   
        2230                             1.599628   
        2258                             7.062072   
        2283                            18.094795   
        2286                             3.578818   

                   teammate_opponent_distance3_a0  \
game_id action_id                                   
3788741 244                              6.905877   
        246                              8.884766   
        373                                   NaN   
        391                              5.283173   
        403                              3.540350   
...                                           ...   
3943043 2207                             4.846302   
        2230                             1.025507   
        2258                             0.977243   
        2283                            14.619251   
        2286                             7.848355   

                   teammate_opponent_distance1_a1  \
game_id action_id                                   
3788741 244                              1.726436   
        246                              2.962817   
        373                                   NaN   
        391                                   NaN   
        403                              0.966707   
...                                           ...   
3943043 2207                             1.105101   
        2230                            13.614290   
        2258                             2.157194   
        2283                            13.963820   
        2286                             9.746009   

                   teammate_opponent_distance2_a1  \
game_id action_id                                   
3788741 244                              6.905877   
        246                             11.497447   
        373                                   NaN   
        391                                   NaN   
        403                              6.882656   
...                                           ...   
3943043 2207                             4.443038   
        2230                             1.469986   
        2258                            15.581577   
        2283                             2.838504   
        2286                             0.958515   

                   teammate_opponent_distance3_a1  \
game_id action_id                                   
3788741 244                              8.566243   
        246                              8.329116   
        373                                   NaN   
        391                                   NaN   
        403                              9.726968   
...                                           ...   
3943043 2207                             1.798191   
        2230                             9.519488   
        2258                           

In [10]:
test_dataset.features

teammate_opponent_distance1_a0  \
game_id action_id                                   
3857254 14                               5.482552   
        17                               4.766174   
        22                               7.255023   
        37                                    NaN   
        90                               1.885806   
...                                           ...   
3869685 2934                             1.634596   
        2937                             4.767050   
        2961                             1.007625   
        2972                             3.141155   
        2993                             2.832621   

                   teammate_opponent_distance2_a0  \
game_id action_id                                   
3857254 14                               9.367999   
        17                               2.582090   
        22                               7.611391   
        37                                    NaN   
        90                               6.241830   
...                                           ...   
3869685 2934                             4.296350   
        2937                             1.749822   
        2961                             3.951134   
        2972                             3.590099   
        2993                             0.482700   

                   teammate_opponent_distance3_a0  \
game_id action_id                                   
3857254 14                               7.525686   
        17                               2.426841   
        22                               4.937432   
        37                                    NaN   
        90                               6.223988   
...                                           ...   
3869685 2934                             0.666180   
        2937                             1.414495   
        2961                             1.019085   
        2972                             7.126078   
        2993                             3.440842   

                   teammate_opponent_distance1_a1  \
game_id action_id                                   
3857254 14                               8.635853   
        17                                    NaN   
        22                              15.673861   
        37                               8.879752   
        90                               2.792802   
...                                           ...   
3869685 2934                             2.794620   
        2937                                  NaN   
        2961                                  NaN   
        2972                                  NaN   
        2993                             3.318753   

                   teammate_opponent_distance2_a1  \
game_id action_id                                   
3857254 14                              10.102416   
        17                                    NaN   
        22                               9.142489   
        37                               7.157477   
        90                               5.284557   
...                                           ...   
3869685 2934                            11.321239   
        2937                                  NaN   
        2961                                  NaN   
        2972                                  NaN   
        2993                             5.266702   

                   teammate_opponent_distance3_a1  \
game_id action_id                                   
3857254 14                               5.308732   
        17                                    NaN   
        22                               9.954089   
        37                               8.987084   
        90                               3.981878   
...                                           ...   
3869685 2934                             3.089821   
        2937                                  NaN   
        2961                           

In [11]:
train_dataset.labels

counterpress
game_id action_id              
3788741 244               False
        246               False
        373               False
        391               False
        403                True
...                         ...
3943043 2207              False
        2230              False
        2258              False
        2283              False
        2286              False

[10945 rows x 1 columns]

In [12]:
test_dataset.labels

counterpress
game_id action_id              
3857254 14                False
        17                False
        22                False
        37                False
        90                 True
...                         ...
3869685 2934              False
        2937              False
        2961              False
        2972              False
        2993              False

[3866 rows x 1 columns]

In [13]:
train_dataset.labels["counterpress"].value_counts()

False    8986
True     1959
Name: counterpress, dtype: int64

In [14]:
test_dataset.labels["counterpress"].value_counts()

False    3087
True      779
Name: counterpress, dtype: int64